# Training Data Generation

In [1]:
import pandas as pd
import constants as c
import utils

from tqdm import tqdm

Define main procedures

In [2]:
def prepare_labels():
    """Load the label descriptor file and preprocess it for merging."""

    df_labels = pd.read_csv(utils.resolve_path(c.DATA_DIR) / c.LABEL_SUBDIR / c.LABEL_FILE,
                            index_col=c.ORBIT_COL,
                            parse_dates=c.EVENT_COLS)

    # disregard incompletely labeled orbits
    df_labels = df_labels.dropna()

    # add columns for predecessor and successor events
    df_labels["Prev out"] = df_labels["SK outer out"].shift()
    df_labels["Next in"] = df_labels["SK outer in"].shift(-1)

    return df_labels

In [3]:
def prepare_orbit(orbit_id):
    """Load the specified orbit and preprocess it for merging."""

    orbit_file = utils.resolve_path(c.DATA_DIR) / c.RAW_SUBDIR / c.ORBIT_FILE(orbit_id)
    df_health.loc[orbit_id, "does_not_exist"] = not orbit_file.exists()

    if not df_health.loc[orbit_id, "does_not_exist"]:
        df_orbit = pd.read_csv(orbit_file, index_col=c.DATE_COL, parse_dates=True)

        # determine validity criteria for orbit
        df_health.loc[orbit_id, "total_nan_count"] = (
                df_orbit.isnull().sum().sum()
        )
        df_health.loc[orbit_id, "missing_entry_count"] = (
                (df_orbit.index[-1] - df_orbit.index[0]).total_seconds() + 1 - len(df_orbit)
        )
        df_health.loc[orbit_id, "has_special_conditions"] = (
                df_labels.loc[orbit_id, "SK outer in"] <= df_orbit.index[0]
                or df_orbit.index[0] <= df_labels.loc[orbit_id, "Prev out"]
                or df_labels.loc[orbit_id, "Next in"] <= df_orbit.index[-1]
                or df_orbit.index[-1] <= df_labels.loc[orbit_id, "SK outer out"]
        )
        df_health.loc[orbit_id, "flux_norm_maximum"] = (
                df_orbit[c.FLUX_COLS].pow(2).sum(axis=1).pow(0.5).max()
        )
        
        if df_health.loc[orbit_id].iloc[0:-1].sum() == 0:
            df_orbit[c.ORBIT_COL] = orbit_id  # add orbit id
            return df_orbit

    return None  # rule out invalid orbit

In [4]:
def assign_labels(df_total, df_labels):
    """Assign labels to the training time series using event boundary labels."""

    # interplanetary magnetic field
    df_total[c.LABEL_COL] = 0

    for row in tqdm(range(len(df_labels))):
        event = lambda col: df_labels.iloc[row, df_labels.columns.get_loc(col)]

        # bow shock crossings
        df_total.loc[event("SK outer in"):event("SK inner in"), c.LABEL_COL] = 1
        df_total.loc[event("SK inner out"):event("SK outer out"), c.LABEL_COL] = 1

        # magnetosheath
        df_total.loc[event("SK inner in"):event("MP outer in"), c.LABEL_COL] = 2
        df_total.loc[event("MP outer out"):event("SK inner out"), c.LABEL_COL] = 2

        # magnetopause crossings
        df_total.loc[event("MP outer in"):event("MP inner in"), c.LABEL_COL] = 3
        df_total.loc[event("MP inner out"):event("MP outer out"), c.LABEL_COL] = 3

        # magnetosphere
        df_total.loc[event("MP inner in"):event("MP inner out"), c.LABEL_COL] = 4

Prepare label descriptor file

In [5]:
df_labels = prepare_labels()
print(f"# validly labeled orbits: {len(df_labels)}")

# validly labeled orbits: 4019


Set up orbit validity statistics

In [6]:
df_health = pd.DataFrame(
    index=df_labels.index,
    columns=["does_not_exist",
             "total_nan_count",
             "missing_entry_count",
             "has_special_conditions",
             "flux_norm_maximum"]
)

Combine orbit data into one big frame

In [7]:
df_total = pd.concat(map(prepare_orbit, tqdm(df_labels.index)))
df_total.iloc[43462:43467]

100%|██████████| 4019/4019 [12:17<00:00,  5.45it/s]


,X_MSO,Y_MSO,Z_MSO,BX_MSO,BY_MSO,BZ_MSO,DBX_MSO,DBY_MSO,DBZ_MSO,RHO_DIPOLE,...,Y,Z,VX,VY,VZ,VABS,D,COSALPHA,EXTREMA,ORBIT
DATE,,,,,,,,,,,,,,,,,,,,,
2011-03-24 19:46:22,6897.941,5403.502,-15374.343,29.343,-2.064,-9.552,0.299,1.637,1.395,18118.125361,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049779,55.749454,4.915513e+07,0.670984,0,2
2011-03-24 19:46:23,6898.235,5403.903,-15374.070,29.540,-1.713,-8.633,0.809,1.914,2.789,18118.117945,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049796,55.749455,4.915513e+07,0.670984,2,2
2011-03-24 19:46:23,6898.235,5403.903,-15374.070,29.540,-1.713,-8.633,0.809,1.914,2.789,18118.117945,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049796,55.749455,4.915513e+07,0.670984,2,3
2011-03-24 19:46:24,6898.530,5404.304,-15373.797,29.849,-0.977,-10.833,1.065,4.469,3.514,18118.110927,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049746,55.749453,4.915513e+07,0.670984,0,3
2011-03-24 19:46:25,6898.825,5404.705,-15373.523,29.333,-2.752,-12.014,1.250,5.516,5.414,18118.103052,...,3.802753e+07,5.912986e+06,-47.808573,-28.603154,2.049724,55.749452,4.915513e+07,0.670984,0,3


Remove extreme outliers by three-sigma rule

In [8]:
df_max = df_health["flux_norm_maximum"]
outlier_orbits = df_health.index[df_max > df_max.mean() + 3 * df_max.std()]
df_total = df_total[~df_total[c.ORBIT_COL].isin(outlier_orbits)]
print(list(outlier_orbits))

[386, 1375, 1615, 1729, 1899, 1946, 3713, 3714, 3718, 4068]


Save orbit validity statistics

In [9]:
df_health.to_csv(utils.resolve_path(c.DATA_DIR) / c.HEALTH_FILE)
orbit_ids = df_total[c.ORBIT_COL].unique()
print(f"# healthy orbits: {len(orbit_ids)}")

# healthy orbits: 3153


Assign labels to the instances of the training data

In [10]:
assign_labels(df_total, df_labels)
df_total.iloc[16605:16610]

100%|██████████| 4019/4019 [5:01:42<00:00,  4.50s/it]  


,X_MSO,Y_MSO,Z_MSO,BX_MSO,BY_MSO,BZ_MSO,DBX_MSO,DBY_MSO,DBZ_MSO,RHO_DIPOLE,...,Z,VX,VY,VZ,VABS,D,COSALPHA,EXTREMA,ORBIT,LABEL
DATE,,,,,,,,,,,,,,,,,,,,,
2011-03-24 12:18:45,5804.272,7081.276,-2428.740,-44.922,16.665,4.784,4.239,6.375,3.369,9608.230719,...,5.855501e+06,-48.713393,-27.442143,2.227680,55.955593,4.894640e+07,0.633532,0,2,0
2011-03-24 12:18:46,5803.633,7080.696,-2427.421,-37.371,8.136,7.607,2.316,3.672,3.443,9607.017439,...,5.855501e+06,-48.713393,-27.442143,2.227663,55.955593,4.894640e+07,0.633532,0,2,0
2011-03-24 12:18:47,5802.993,7080.117,-2426.101,-34.104,4.656,14.430,1.949,0.970,1.545,9605.804096,...,5.855501e+06,-48.713393,-27.442143,2.227660,55.955593,4.894640e+07,0.633532,0,2,1
2011-03-24 12:18:48,5802.353,7079.538,-2424.782,-37.385,-0.449,23.100,1.797,3.131,4.607,9604.591160,...,5.855501e+06,-48.713393,-27.442143,2.227669,55.955593,4.894640e+07,0.633532,0,2,1
2011-03-24 12:18:49,5801.714,7078.958,-2423.462,-31.174,2.786,35.948,3.817,7.081,2.152,9603.377895,...,5.855501e+06,-48.713393,-27.442143,2.227612,55.955591,4.894640e+07,0.633532,0,2,1


Save class distribution information

In [11]:
df_freqs = df_total.groupby(c.LABEL_COL).size()
df_freqs.to_csv(utils.resolve_path(c.DATA_DIR) / c.FREQS_FILE)
df_freqs

LABEL
0    64868418
1     3667178
2    14346807
3     2286557
4    13957973
dtype: int64

Partition data into $90\%$ training and $10\%$ testing set

In [12]:
test_orbits = pd.Series(orbit_ids).sample(len(orbit_ids) // 10)
df_train = df_total[~df_total[c.ORBIT_COL].isin(test_orbits)]
df_test = df_total[df_total[c.ORBIT_COL].isin(test_orbits)]

Save descriptive statistics about the training set

In [13]:
df_stats = df_train.describe()
df_stats.index.name = c.STAT_COL
df_stats.to_csv(utils.resolve_path(c.DATA_DIR) / c.STATS_FILE)
df_stats

,X_MSO,Y_MSO,Z_MSO,BX_MSO,BY_MSO,BZ_MSO,DBX_MSO,DBY_MSO,DBZ_MSO,RHO_DIPOLE,...,Z,VX,VY,VZ,VABS,D,COSALPHA,EXTREMA,ORBIT,LABEL
STAT,,,,,,,,,,,,,,,,,,,,,
count,8.917289e+07,8.917289e+07,8.917289e+07,8.917289e+07,8.917289e+07,8.917289e+07,8.917289e+07,8.917289e+07,8.917289e+07,8.917289e+07,...,8.917289e+07,8.917289e+07,8.917289e+07,8.917289e+07,8.917289e+07,8.917289e+07,8.917289e+07,8.917289e+07,8.917289e+07,8.917289e+07
mean,1.315765e+02,-1.639584e+02,-7.899505e+03,4.337857e-01,-2.653793e-01,-4.454915e+00,1.060072e+00,1.400845e+00,1.399623e+00,1.051222e+04,...,-1.477714e+06,-2.943107e-02,3.703317e+00,3.053322e-01,4.752608e+01,5.895149e+07,2.349638e-03,6.366285e-05,1.862997e+03,9.604381e-01
std,3.665622e+03,3.885022e+03,5.242040e+03,4.071519e+01,3.999325e+01,5.646241e+01,1.653099e+00,2.094693e+00,2.100094e+00,3.978662e+03,...,4.885600e+06,3.417986e+01,3.331217e+01,4.125495e+00,7.075591e+00,8.439905e+06,6.874760e-01,2.111009e-02,1.169493e+03,1.469557e+00
min,-1.074348e+04,-1.131478e+04,-1.662732e+04,-3.955740e+02,-5.540960e+02,-5.791980e+02,0.000000e+00,0.000000e+00,0.000000e+00,2.015986e+03,...,-7.654416e+06,-5.848620e+01,-4.661918e+01,-4.894014e+00,3.885789e+01,4.599997e+07,-1.000000e+00,-2.000000e+00,2.000000e+00,0.000000e+00
25%,-2.319364e+03,-2.806297e+03,-1.196499e+04,-1.751600e+01,-1.121700e+01,-8.230000e+00,1.800000e-01,2.150000e-01,2.370000e-01,7.759019e+03,...,-6.329423e+06,-3.249120e+01,-2.789521e+01,-3.679371e+00,4.065040e+01,5.028207e+07,-6.630508e-01,0.000000e+00,8.180000e+02,0.000000e+00
50%,-8.294400e+01,-7.430100e+01,-9.311913e+03,-2.480000e-01,-5.300000e-02,5.370000e-01,5.130000e-01,6.970000e-01,7.100000e-01,1.137309e+04,...,-2.101660e+06,9.672237e+00,5.127849e+00,-4.220127e-01,4.599979e+01,6.021827e+07,1.899174e-03,0.000000e+00,1.790000e+03,0.000000e+00
75%,2.554231e+03,2.530306e+03,-4.026452e+03,1.556500e+01,1.101100e+01,9.863000e+00,1.286000e+00,1.775000e+00,1.751000e+00,1.302737e+04,...,3.339362e+06,3.231478e+01,3.599440e+01,4.215396e+00,5.465329e+01,6.729511e+07,6.773884e-01,0.000000e+00,2.794000e+03,2.000000e+00
max,1.142345e+04,1.111089e+04,3.154425e+03,5.589880e+02,5.206690e+02,3.956680e+02,3.332820e+02,2.960840e+02,2.866980e+02,1.812033e+04,...,6.239501e+06,3.894385e+01,5.089473e+01,7.036499e+00,5.897771e+01,6.981782e+07,9.999954e-01,2.000000e+00,4.092000e+03,4.000000e+00


Save the final training data frames

In [14]:
train_dir = utils.resolve_path(c.DATA_DIR) / c.TRAIN_SUBDIR
utils.ensure_directory(train_dir)

for n, orbit in tqdm(df_train.groupby(c.ORBIT_COL)):
    orbit.to_csv(train_dir / c.ORBIT_FILE(n))

100%|██████████| 2838/2838 [36:08<00:00,  1.31it/s]


Save the final testing data frames

In [15]:
test_dir = utils.resolve_path(c.DATA_DIR) / c.TEST_SUBDIR
utils.ensure_directory(test_dir)

for n, orbit in tqdm(df_test.groupby(c.ORBIT_COL)):
    orbit.to_csv(test_dir / c.ORBIT_FILE(n))

100%|██████████| 315/315 [04:02<00:00,  1.30it/s]
